In [2]:
import cloudvolume
from cloudvolume import CloudVolume, Bbox
import numpy as np

sem = CloudVolume('s3://bossdb-open-data/witvliet2020/Dataset_5/em/', mip=4, secrets=None, use_https=True,fill_missing=True)
sem_labels = CloudVolume('s3://bossdb-open-data/witvliet2020/Dataset_5/segmentation/', mip=4, secrets=None, use_https=True,fill_missing=True)
tem = CloudVolume('s3://bossdb-open-data/witvliet2020/Dataset_6/em/', secrets=None, mip=4, use_https=True,fill_missing=True)
tem_labels = CloudVolume('s3://bossdb-open-data/witvliet2020/Dataset_6/segmentation/', mip=4, secrets=None, use_https=True,fill_missing=True)
sem.refresh_info()
sem_labels.refresh_info()
tem.refresh_info()
tem_labels.refresh_info()



{'data_type': 'uint64',
 'num_channels': 1,
 'scales': [{'chunk_sizes': [[512, 512, 16]],
   'encoding': 'raw',
   'key': '0.768_0.768_50.0',
   'resolution': [0.768, 0.768, 50.0],
   'size': [37376, 35328, 448],
   'voxel_offset': [0, 0, 0]},
  {'chunk_sizes': [[512, 512, 16]],
   'encoding': 'raw',
   'key': '1.536_1.536_50.0',
   'resolution': [1.536, 1.536, 50.0],
   'size': [18688, 17664, 448],
   'voxel_offset': [0, 0, 0]},
  {'chunk_sizes': [[512, 512, 16]],
   'encoding': 'raw',
   'key': '3.072_3.072_50.0',
   'resolution': [3.072, 3.072, 50.0],
   'size': [9344, 8832, 448],
   'voxel_offset': [0, 0, 0]},
  {'chunk_sizes': [[512, 512, 16]],
   'encoding': 'raw',
   'key': '6.144_6.144_50.0',
   'resolution': [6.144, 6.144, 50.0],
   'size': [4672, 4416, 448],
   'voxel_offset': [0, 0, 0]},
  {'chunk_sizes': [[512, 512, 16]],
   'encoding': 'raw',
   'key': '12.288_12.288_50.0',
   'resolution': [12.288, 12.288, 50.0],
   'size': [2336, 2208, 448],
   'voxel_offset': [0, 0, 0]}

In [3]:
import zarr

# Assert that the Zarr version is 2.*
assert zarr.__version__.startswith('2.'), f"Zarr version must be 2: {zarr.__version__}"


In [4]:
print("---- SEM (nm) ----")
# Print resolutions of sem at all mip levels
for mip_level in range(len(sem.scales)):
    resolution = sem.scales[mip_level]['resolution']
    print(f"Resolution at mip={mip_level}: {resolution}")

print("---- SEM Labels (nm) ----")
# Print resolutions of sem_labels at all mip levels
for mip_level in range(len(sem_labels.scales)):
    resolution = sem_labels.scales[mip_level]['resolution']
    print(f"Resolution at mip={mip_level}: {resolution}")

print("---- TEM (nm) ----")
# Print resolutions of tem at all mip levels
for mip_level in range(len(tem.scales)):
    resolution = tem.scales[mip_level]['resolution']
    print(f"Resolution at mip={mip_level}: {resolution}")

print("---- TEM Labels (nm) ----")
# Print resolutions of tem_labels at all mip levels
for mip_level in range(len(tem_labels.scales)):
    resolution = tem_labels.scales[mip_level]['resolution']
    print(f"Resolution at mip={mip_level}: {resolution}")


---- SEM (nm) ----
Resolution at mip=0: [2.0, 2.0, 30.0]
Resolution at mip=1: [4.0, 4.0, 30.0]
Resolution at mip=2: [8.0, 8.0, 30.0]
Resolution at mip=3: [16.0, 16.0, 30.0]
Resolution at mip=4: [32.0, 32.0, 30.0]
Resolution at mip=5: [64.0, 64.0, 30.0]
---- SEM Labels (nm) ----
Resolution at mip=0: [2.0, 2.0, 30.0]
Resolution at mip=1: [4.0, 4.0, 30.0]
Resolution at mip=2: [8.0, 8.0, 30.0]
Resolution at mip=3: [16.0, 16.0, 30.0]
Resolution at mip=4: [32.0, 32.0, 30.0]
---- TEM (nm) ----
Resolution at mip=0: [0.768, 0.768, 50.0]
Resolution at mip=1: [1.536, 1.536, 50.0]
Resolution at mip=2: [3.072, 3.072, 50.0]
Resolution at mip=3: [6.144, 6.144, 50.0]
Resolution at mip=4: [12.288, 12.288, 50.0]
Resolution at mip=5: [24.576, 24.576, 50.0]
Resolution at mip=6: [49.152, 49.152, 50.0]
---- TEM Labels (nm) ----
Resolution at mip=0: [0.768, 0.768, 50.0]
Resolution at mip=1: [1.536, 1.536, 50.0]
Resolution at mip=2: [3.072, 3.072, 50.0]
Resolution at mip=3: [6.144, 6.144, 50.0]
Resolution at 

In [5]:
print(f"Bounds: {sem.bounds}")

Bounds: Bbox([np.int32(0), np.int32(0), np.int32(0)],[np.int32(864), np.int32(704), np.int32(864)], dtype=np.int32, unit='vx')


In [6]:
import os
import numpy as np
import zarr
from cloudvolume import CloudVolume, Bbox
import zarr.storage

def download_and_save_as_zarr(em_volume: CloudVolume, seg_volume: CloudVolume, save_path: str, mip_level=4, squeeze_c_dim=True):
    """
    Downloads two cloud volumes (EM and segmentation) at their bounding boxes,
    finds the intersection if the bounding boxes are mismatched, and saves
    the resulting subvolume data in a Zarr directory store (with .zarray, .zgroup, etc.).

    Parameters
    ----------
    em_volume : CloudVolume
        The CloudVolume instance for the EM data to download.
    seg_volume : CloudVolume
        The CloudVolume instance for the segmentation data to download.
    save_path : str
        The path to the directory where the Zarr store will be saved.
    """
    # 1) Get full bounding boxes for EM and segmentation
    em_bbox = Bbox.from_delta(em_volume.bounds.minpt, em_volume.bounds.size3())
    seg_bbox = Bbox.from_delta(seg_volume.bounds.minpt, seg_volume.bounds.size3())

    # 2) Calculate the intersection of the bounding boxes
    intersection_bbox = em_bbox.intersection(em_bbox, seg_bbox)

    # 3) Download data from the intersection region
    em_data = em_volume.download(intersection_bbox)
    seg_data = seg_volume.download(intersection_bbox)

    # 4) Convert to numpy arrays if necessary and print dtype and check if not all zero
    if not isinstance(em_data, np.ndarray):
        em_data = np.array(em_data)
    print(f"EM data dtype: {em_data.dtype}")
    if np.any(em_data != 0):
        print("EM data is not all zero.")
    
    if not isinstance(seg_data, np.ndarray):
        seg_data = np.array(seg_data)
    print(f"Segmentation data dtype: {seg_data.dtype}")
    if np.any(seg_data != 0):
        print("Segmentation data is not all zero. ✅")

    # 5) Create a standard Zarr directory store (which includes .zarray, .zgroup, etc.)
    store = zarr.DirectoryStore(save_path)
    root = zarr.group(store=store, overwrite=True)
    
    print("Creating arrays")
    
    # 6) Create datasets with chunked layout (adjust chunks as desired)
    # Adjust chunk sizes to avoid exceeding buffer size limits

    # Assert that the channel axis is 1 and remove it if present, only if ndim is 4 and squeeze_c is True
    if squeeze_c_dim and em_data.ndim == 4:
        assert em_data.shape[-1] == 1, "EM data channel axis is not 1"
        em_data = em_data.squeeze(axis=-1)
    
    if squeeze_c_dim and seg_data.ndim == 4:
        assert seg_data.shape[-1] == 1, "Segmentation data channel axis is not 1"
        seg_data = seg_data.squeeze(axis=-1)

    raw_array = root.create_dataset('raw', data=em_data, shape=em_data.shape, dtype=em_data.dtype)
    seg_array = root.create_dataset('gt_labels', data=seg_data, shape=seg_data.shape, dtype=seg_data.dtype)
    
    # 7) Extract metadata for the specified mip level
    em_info = em_volume.info['scales'][mip_level]
    seg_info = seg_volume.info['scales'][mip_level]
    
    # 8) Create .zattrs for metadata using extracted values
    raw_array.attrs.update({
        'description': 'Raw electron microscopy data',
        'encoding': em_info['encoding'],
        'offset': em_info['voxel_offset'],
        'axis_names': ['x', 'y', 'z'],
        'units': ['nm', 'nm', 'nm'],
        'voxel_size': em_info['resolution']
    })
    seg_array.attrs.update({
        'description': 'Ground truth labels for segmentation',
        'encoding': seg_info['encoding'],
        'offset': seg_info['voxel_offset'],
        'axis_names': ['x', 'y', 'z'],
        'units': ['nm', 'nm', 'nm'],
        'voxel_size': seg_info['resolution']
    })

    print(f"Zarr datasets saved in '{save_path}'.")

In [7]:
import os
to_download = [[sem, sem_labels], [tem, tem_labels]]
zarrs = ["sem.zarr", "tem.zarr"]
break_array = [False, False]
base_download_path = "data_neuroglancer"


for (em,labels), target_path, toggle_break in zip(to_download,zarrs,break_array):
    if not toggle_break:
        print(f"Downloading and saving to {target_path}")
        download_and_save_as_zarr(em,labels,save_path=os.path.join(base_download_path,target_path),squeeze_c_dim=True)   

Downloading: 100%|██████████| 216/216 [00:12<00:00, 17.00it/s]


EM data dtype: uint8
EM data is not all zero.
Segmentation data dtype: uint64
Segmentation data is not all zero. ✅
Creating arrays
Zarr datasets saved in 'data_neuroglancer/sem.zarr'.


Downloading: 100%|██████████| 675/675 [00:15<00:00, 44.13it/s] 


EM data dtype: uint8
EM data is not all zero.
Segmentation data dtype: uint64
Segmentation data is not all zero. ✅
Creating arrays
Zarr datasets saved in 'data_neuroglancer/tem.zarr'.


In [51]:
import zarr
import numpy as np
import os 
def reorder_xyz_to_zyx(zarr_path):
    print(f"Opening Zarr store at path: {zarr_path}")
    store = zarr.open(zarr_path, mode='r+')
    
    # recursively handle groups
    def _reorder_group(group):
        print(f"Processing group: {group.name}")
        for k, v in group.items():
            if isinstance(v, zarr.Group):
                print(f"Found subgroup: {k}")
                _reorder_group(v)
            elif isinstance(v, zarr.Array):
                print(f"Found array: {k}")
                axes = v.attrs.get('axis_names', None)
                print(f"Current axes: {axes}")
                if axes == ['x', 'y', 'z']:
                    print(f"Reordering array: {k}")
                    data = v[...]
                    # reorder
                    data = np.transpose(data, (2,1,0))
                    # update data & axes
                    v.resize(data.shape)
                    v[...] = data
                    v.attrs['axis_names'] = ['z', 'y', 'x']
                    v.attrs['offset'] = [v.attrs['offset'][2], v.attrs['offset'][1], v.attrs['offset'][0]]
                    v.attrs['voxel_size'] = [v.attrs['voxel_size'][2], v.attrs['voxel_size'][1], v.attrs['voxel_size'][0]]
                    v.attrs['units'] = [v.attrs['units'][2], v.attrs['units'][1], v.attrs['units'][0]]
                    v.attrs['axis_names'] = ['z', 'y', 'x']
                    print(f"Reordered axes: {v.attrs['axis_names']}")
    
    _reorder_group(store)
    print(f"Completed reordering for Zarr store at path: {zarr_path}")

In [52]:
for zarr_path in ["sem.zarr", "tem.zarr"]:
    reorder_xyz_to_zyx(os.path.join("data",zarr_path))

Opening Zarr store at path: data/sem.zarr
Processing group: /
Found array: gt_labels
Current axes: ['x', 'y', 'z']
Reordering array: gt_labels
Reordered axes: ['z', 'y', 'x']
Found array: raw
Current axes: ['x', 'y', 'z']
Reordering array: raw
Reordered axes: ['z', 'y', 'x']
Completed reordering for Zarr store at path: data/sem.zarr
Opening Zarr store at path: data/tem.zarr
Processing group: /
Found array: gt_labels
Current axes: ['x', 'y', 'z']
Reordering array: gt_labels
Reordered axes: ['z', 'y', 'x']
Found array: raw
Current axes: ['x', 'y', 'z']
Reordering array: raw
Reordered axes: ['z', 'y', 'x']
Completed reordering for Zarr store at path: data/tem.zarr


In [53]:
print("Success downloading and reordering all datasets!")

Success downloading and reordering all datasets!
